# Evaluación - Redes Convolucionales

**Autor:**

**Correo Electrónico:**

**Fecha de Creación:** Mayo de 2025

**Versión:** 1.0  

---

## Descripción

Este notebook contiene el desarrollo de la entrega 2 de la asignatura optativa de Deep Learning de DuocUC Sede Concepción

---

## Requisitos de Software

Este notebook fue desarrollado con Python 3.9. A continuación se listan las bibliotecas necesarias:

- tensorflow (2.18.0)

Para verificar la versión instalada ejecutar usando el siguiente comando, usando la librería de la cual quieres saber la versión:

```bash
import tensorflow as tf
print(tf.__version__)
````

# Entregable
*   Comparación de modelos CNNs con un modelo de Fully Connected para este problema.
* No es necesario mostrar en el notebook las trazas de entrenamiento de todos los modelos entrenados, si bien una buena idea seria guardar gráficas de esos entrenamientos para el análisis. Sin embargo, **se debe mostrar el entrenamiento completo del mejor modelo obtenido y la evaluación de los datos de test con este modelo**.

# Liberías

In [52]:
# Librerías básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import collections
import pprint
import os
import glob

# Librerías para procesamiento y visualización de imágenes
import cv2
from google.colab.patches import cv2_imshow

# TensorFlow y Keras (CNN)
import tensorflow as tf
from tensorflow.keras import layers, Sequential, Model
from tensorflow.keras.layers import (
    Input, Dense, Activation, Dropout, Flatten, BatchNormalization,
    Conv2D, MaxPooling2D, RandomFlip, RandomRotation, RandomZoom
)
from tensorflow.keras.optimizers import Adam, SGD, Adadelta
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.utils import to_categorical
from IPython.display import display

# Librerías de Scikit-learn para métricas y división de datos
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split

# Configuración gráfica para matplotlib en Colab
%matplotlib inline

# Descarga de la data

* `!wget`: utilidad de línea de comandos para descargar archivos desde una URL.

* `-O simpsons_train.tar.gz`: fuerza a guardar el archivo descargado con ese nombre en el disco local.

La URL de Dropbox es el enlace compartido que contiene el tarball con todas las imágenes del conjunto de entrenamiento de Simpsons.

In [2]:
!wget -O simpsons_train.tar.gz https://www.dropbox.com/scl/fi/qkg3gs31xjbhv9jjqmot6/simpsons_train.tar.gz?rlkey=oqbofdqoqjrpmxjwxaphru0yr&st=b96sg8iu&dl=0

--2025-05-18 01:37:53--  https://www.dropbox.com/scl/fi/qkg3gs31xjbhv9jjqmot6/simpsons_train.tar.gz?rlkey=oqbofdqoqjrpmxjwxaphru0yr
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc97534ce7f9123b50a7d6f5b328.dl.dropboxusercontent.com/cd/0/inline/Cp6P4dBp7jy9aVFvB2FdK9xC9Io5FQFMYdlnF6bNEGSTd_zZpeAVLpnRUK66P9jwHyu_aB-i5Bfs0zhcGRAnJb9kKNbKlJxoWi8w4wr_dIMomUNxyf2gafDtAZ9hMMfGrIvmZqclPTyHEbnnR7-YVIE1/file# [following]
--2025-05-18 01:37:54--  https://uc97534ce7f9123b50a7d6f5b328.dl.dropboxusercontent.com/cd/0/inline/Cp6P4dBp7jy9aVFvB2FdK9xC9Io5FQFMYdlnF6bNEGSTd_zZpeAVLpnRUK66P9jwHyu_aB-i5Bfs0zhcGRAnJb9kKNbKlJxoWi8w4wr_dIMomUNxyf2gafDtAZ9hMMfGrIvmZqclPTyHEbnnR7-YVIE1/file
Resolving uc97534ce7f9123b50a7d6f5b328.dl.dropboxusercontent.com (uc97534ce7f9123b50a7d6f5b328.dl.dropboxusercontent.com)... 162.125.66.

Similar al paso anterior, pero apuntando al archivo que contiene las imágenes de test.

In [3]:
!wget -O simpsons_test.tar.gz https://www.dropbox.com/scl/fi/zche5dm3zgd9jysatnmka/simpsons_test.tar.gz?rlkey=iek183gc4t4w9mdnz1izhudni&st=qau98qns&dl=0

--2025-05-18 01:38:29--  https://www.dropbox.com/scl/fi/zche5dm3zgd9jysatnmka/simpsons_test.tar.gz?rlkey=iek183gc4t4w9mdnz1izhudni
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8f1f6b8c4e803810eb45cfd4e1.dl.dropboxusercontent.com/cd/0/inline/Cp5RFXKypHgI6-2_kKFPDXaYyN2MJJ6fY8YogHSkzvoyzjF4XJz3JmWiHUCfHh3isvdkzXiTsGFiBuGZDKhyVDXrNDu_JEUIy-JAyGumE8Z6B4ZzBWB5X7Kw8yXxrOfR0bAclh_XW9j5qQHFtZQDB3rt/file# [following]
--2025-05-18 01:38:29--  https://uc8f1f6b8c4e803810eb45cfd4e1.dl.dropboxusercontent.com/cd/0/inline/Cp5RFXKypHgI6-2_kKFPDXaYyN2MJJ6fY8YogHSkzvoyzjF4XJz3JmWiHUCfHh3isvdkzXiTsGFiBuGZDKhyVDXrNDu_JEUIy-JAyGumE8Z6B4ZzBWB5X7Kw8yXxrOfR0bAclh_XW9j5qQHFtZQDB3rt/file
Resolving uc8f1f6b8c4e803810eb45cfd4e1.dl.dropboxusercontent.com (uc8f1f6b8c4e803810eb45cfd4e1.dl.dropboxusercontent.com)... 162.125.66.1

* `tar`: comando para trabajar con archivos tar (archivos empaquetados).

* Flags utilizados:

  * `-x` → extraer.

  * `-z` → descomprimir gzip.

  * `-v` → modo verboso (muestra en pantalla cada fichero que va extrayendo).

  * `-f` → indica el nombre del archivo tar a procesar.

Tras ejecutarlo, se crea una carpeta (llamada `simpsons/`) con todas las subcarpetas o imágenes descomprimidas.

In [4]:
!tar -xzvf simpsons_train.tar.gz

Se han truncado las últimas 5000 líneas del flujo de salida.
simpsons/mayor_quimby/pic_0116.jpg
simpsons/milhouse_van_houten/pic_0576.jpg
simpsons/lenny_leonard/pic_0149.jpg
simpsons/kent_brockman/pic_0446.jpg
simpsons/nelson_muntz/pic_0060.jpg
simpsons/krusty_the_clown/pic_0838.jpg
simpsons/homer_simpson/pic_0637.jpg
simpsons/homer_simpson/pic_0495.jpg
simpsons/krusty_the_clown/pic_0893.jpg
simpsons/homer_simpson/pic_0834.jpg
simpsons/homer_simpson/pic_0692.jpg
simpsons/lisa_simpson/pic_0755.jpg
simpsons/marge_simpson/pic_0654.jpg
simpsons/chief_wiggum/pic_0344.jpg
simpsons/lisa_simpson/pic_0952.jpg
simpsons/lisa_simpson/pic_1269.jpg
simpsons/marge_simpson/pic_0851.jpg
simpsons/marge_simpson/pic_1168.jpg
simpsons/chief_wiggum/pic_0541.jpg
simpsons/homer_simpson/pic_1948.jpg
simpsons/sideshow_bob/pic_0104.jpg
simpsons/lisa_simpson/pic_0278.jpg
simpsons/nelson_muntz/pic_0128.jpg
simpsons/marge_simpson/pic_0177.jpg
simpsons/milhouse_van_houten/pic_0699.jpg
simpsons/waylon_smithers/pic_00

Igual que en el paso anterior, pero se extrae el tarball de test, generando la carpeta `simpsons_testset/`.

In [5]:
!tar -xzvf simpsons_test.tar.gz

simpsons_testset/charles_montgomery_burns_46.jpg
simpsons_testset/marge_simpson_35.jpg
simpsons_testset/abraham_grampa_simpson_9.jpg
simpsons_testset/krusty_the_clown_40.jpg
simpsons_testset/apu_nahasapeemapetilon_10.jpg
simpsons_testset/homer_simpson_24.jpg
simpsons_testset/lenny_leonard_17.jpg
simpsons_testset/marge_simpson_3.jpg
simpsons_testset/milhouse_van_houten_13.jpg
simpsons_testset/apu_nahasapeemapetilon_29.jpg
simpsons_testset/lenny_leonard_34.jpg
simpsons_testset/bart_simpson_27.jpg
simpsons_testset/ned_flanders_12.jpg
simpsons_testset/charles_montgomery_burns_3.jpg
simpsons_testset/milhouse_van_houten_30.jpg
simpsons_testset/apu_nahasapeemapetilon_46.jpg
simpsons_testset/mayor_quimby_3.jpg
simpsons_testset/kent_brockman_6.jpg
simpsons_testset/chief_wiggum_25.jpg
simpsons_testset/nelson_muntz_37.jpg
simpsons_testset/moe_szyslak_28.jpg
simpsons_testset/bart_simpson_44.jpg
simpsons_testset/homer_simpson_36.jpg
simpsons_testset/milhouse_van_houten_49.jpg
simpsons_testset/comic

Las dos funciones presentadas sirven para leer, redimensionar y etiquetar las imágenes del proyecto antes de alimentar la red neuronal. A continuación se detalla su comportamiento en cuatro apartados comunes:


Propósito general:

  * `load_train_set`: carga el conjunto de entrenamiento desde carpetas organizadas por personaje.
  * `load_test_set`: carga el conjunto de prueba desde un directorio con imágenes sueltas, inferiendo la etiqueta a partir del nombre de archivo.

Proceso:

* Se crean dos listas vacías (una para imágenes y otra para etiquetas).

* Recorrido de archivos:
  * En `load_train_set` se itera sobre cada par `(etiqueta, personaje)` de `map_characters` y se listan los ficheros de imagen dentro de la carpeta correspondiente.
  * En `load_test_set` se recorre todo el directorio con un patrón genérico (`glob`) y se extrae el nombre del personaje de cada nombre de archivo, buscando el prefijo antes del último guion bajo.
* Lectura y redimensionado:
  * Cada imagen se lee con OpenCV (`cv2.imread`) y se aplica `cv2.resize` para ajustar la imagen a (`IMG_SIZE, IMG_SIZE`) usando interpolación adecuada.

* Etiquetado
  * En entrenamiento la etiqueta viene directamente de la clave del diccionario y en prueba: se invierte el diccionario para mapear el nombre de personaje extraído al índice numérico.

Al finalizar el bucle, las listas de imágenes y etiquetas se convierten a `np.ndarray` con el tipo de dato apropiado (`float32` para imágenes, `int32` para etiquetas).



In [6]:
def load_train_set(dirname : str, map_characters : np.array, verbose=True):
    """Esta función carga los datos de training en imágenes.

    Como las imágenes tienen tamaños distintas, utilizamos la librería opencv
    para hacer un resize y adaptarlas todas a tamaño IMG_SIZE x IMG_SIZE.

    Args:
        dirname: directorio completo del que leer los datos
        map_characters: variable de mapeo entre labels y personajes
        verbose: si es True, muestra información de las imágenes cargadas

    Returns:
        X, y: X es un array con todas las imágenes cargadas con tamaño
                IMG_SIZE x IMG_SIZE
              y es un array con las labels de correspondientes a cada imagen
    """
    X_train = []
    y_train = []
    for label, character in map_characters.items():
        files = os.listdir(os.path.join(dirname, character))
        images = [file for file in files if file.endswith("jpg")]
        if verbose:
          print("Leyendo {} imágenes encontradas de {}".format(len(images), character))
        for image_name in images:
            image = cv2.imread(os.path.join(dirname, character, image_name))
            X_train.append(cv2.resize(image,(IMG_SIZE, IMG_SIZE)))
            y_train.append(label)
    return np.array(X_train), np.array(y_train)

def load_test_set(dirname : str, map_characters : np.array, verbose=True):
    """Esta función funciona de manera equivalente a la función load_train_set
    pero cargando los datos de test."""
    X_test = []
    y_test = []
    reverse_dict = {v: k for k, v in map_characters.items()}
    for filename in glob.glob(dirname + '/*.*'):
        char_name = "_".join(filename.split('/')[-1].split('_')[:-1])
        if char_name in reverse_dict:
            image = cv2.imread(filename)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            X_test.append(image)
            y_test.append(reverse_dict[char_name])
    if verbose:
        print("Leídas {} imágenes de test".format(len(X_test)))
    return np.array(X_test), np.array(y_test)

Es un diccionario que asocia cada etiqueta numérica (clave) con el nombre del personaje (valor) correspondiente.

`IMG_SIZE` Define el tamaño fijo (en píxeles) al que se redimensionarán todas las imágenes antes de pasarlas a la red neuronal. En este caso, `IMG_SIZE = 64` implica que cada imagen se transformará a un cuadrado de 64 × 64 píxeles.

In [7]:
# Esta variable contiene un mapeo de número de clase a personaje.
# Se utilizan sólo los 18 personajes del dataset que tienen más imágenes.
MAP_CHARACTERS = {
    0: 'abraham_grampa_simpson', 1: 'apu_nahasapeemapetilon', 2: 'bart_simpson',
    3: 'charles_montgomery_burns', 4: 'chief_wiggum', 5: 'comic_book_guy', 6: 'edna_krabappel',
    7: 'homer_simpson', 8: 'kent_brockman', 9: 'krusty_the_clown', 10: 'lisa_simpson',
    11: 'marge_simpson', 12: 'milhouse_van_houten', 13: 'moe_szyslak',
    14: 'ned_flanders', 15: 'nelson_muntz', 16: 'principal_skinner', 17: 'sideshow_bob'
}

# Se estandarizan todas las imágenes a tamaño 64x64
IMG_SIZE = 64

Se asignan las carpetas `"simpsons"` (entrenamiento) y `"simpsons_testset"` (prueba) a las variables `DATASET_TRAIN_PATH_COLAB` y `DATASET_TEST_PATH_COLAB` respectivamente, y a continuación se llaman las funciones `load_train_set` y `load_test_set` con estas rutas y el diccionario `MAP_CHARACTERS` para cargar en memoria los arrays `X, y` (imágenes y etiquetas de entrenamiento) y `X_test, y_test` (imágenes y etiquetas de prueba).

In [8]:
# Carga los datos
DATASET_TRAIN_PATH_COLAB = "simpsons"
DATASET_TEST_PATH_COLAB = "simpsons_testset"

X, y = load_train_set(DATASET_TRAIN_PATH_COLAB, MAP_CHARACTERS)
X_test, y_test = load_test_set(DATASET_TEST_PATH_COLAB, MAP_CHARACTERS)

Leyendo 913 imágenes encontradas de abraham_grampa_simpson
Leyendo 623 imágenes encontradas de apu_nahasapeemapetilon
Leyendo 1342 imágenes encontradas de bart_simpson
Leyendo 1193 imágenes encontradas de charles_montgomery_burns
Leyendo 986 imágenes encontradas de chief_wiggum
Leyendo 469 imágenes encontradas de comic_book_guy
Leyendo 457 imágenes encontradas de edna_krabappel
Leyendo 2246 imágenes encontradas de homer_simpson
Leyendo 498 imágenes encontradas de kent_brockman
Leyendo 1206 imágenes encontradas de krusty_the_clown
Leyendo 1354 imágenes encontradas de lisa_simpson
Leyendo 1291 imágenes encontradas de marge_simpson
Leyendo 1079 imágenes encontradas de milhouse_van_houten
Leyendo 1452 imágenes encontradas de moe_szyslak
Leyendo 1454 imágenes encontradas de ned_flanders
Leyendo 358 imágenes encontradas de nelson_muntz
Leyendo 1194 imágenes encontradas de principal_skinner
Leyendo 877 imágenes encontradas de sideshow_bob
Leídas 890 imágenes de test


Se genera un array `perm` con una permutación aleatoria de los índices de `X`, y a continuación se reordenan simultáneamente `X` e `y` usando esa permutación, de modo que las imágenes y sus etiquetas queden barajadas aleatoriamente antes de cualquier partición o entrenamiento.

In [9]:
# Se va a barajar aleatoriamente los datos. Esto es importante ya que si no
# se realiza y, por ejemplo, se escogen el 20% de los datos finales como validation
# set, se estará utilizando solo un pequeño número de personajes, ya que
# las imágenes se leen secuencialmente personaje a personaje.
perm = np.random.permutation(len(X))
X, y = X[perm], y[perm]

Se utiliza `train_test_split` para reservar el 20 % de las muestras originales `X, y` como conjunto de validación (`X_val`, `y_val`), mientras que el 80 % restante conforma `X_train, y_train`. A continuación, se extraen los tamaños de cada partición (`X_train_num_elem`, `X_val_num_elem`, `X_test_num_elem`) y se definen variables que describen la resolución (`img_rows`, `img_cols`), el número de canales (`img_channels`) y la cantidad de clases (`num_categ`).

Finalmente, se imprimen en consola tanto el número de imágenes y etiquetas en cada subconjunto como sus dimensiones (número de muestras y forma de cada array), de modo que podamos verificar de un vistazo que la división y los parámetros son los esperados antes de continuar con la normalización y el entrenamiento del modelo.

In [10]:
# Separa el conjunto de datos de entrenamiento en entrenamiento y validación con 20%
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=29)

# Variables con nÚmero de elementos
X_train_num_elem = X_train.shape[0]
X_val_num_elem = X_val.shape[0]
X_test_num_elem = X_test.shape[0]

# Dimensiones
img_rows = IMG_SIZE
img_cols = IMG_SIZE
img_channels = X.shape[3]

# Número de categorías
num_categ = len(MAP_CHARACTERS)

# Comprueba que efectivamente los resultados son los esperados
print("Número de imágenes de anterior training : ", X.shape[0])
print("Número de imágenes de training (80% de anterior train) : ", X_train_num_elem)
print("Número de imágenes de validación (20% de anterior train) : ", X_val_num_elem)
print("Número de imágenes de test : ", X_test_num_elem)
print("="*15)
print("Número de resultados de anterior training : ", y.shape[0])
print("Número de resultados de training (80% de anterior train) : ", y_train.shape[0])
print("Número de resultadoss de validación (20% de anterior train) : ", y_val.shape[0])
print("Número de resultados de test : ", y_test.shape[0])
print("="*15)
print("Dimensiones de training : ", X_train.shape)
print("Dimensiones de validation : ", X_val.shape)
print("Dimensiones de tests : ", X_test.shape)
print("="*15)
print("Canales de color para las imagenes : ", img_channels)
print("Resolución imágenes de anterior training : ", X.shape[1], "x", X.shape[2])
print("Resolución imágenes de training (80% de anterior train) : ", X_train.shape[1], "X", X_train.shape[2])
print("Resolución imágenes de validación (20% de anterior train) : ", X_val.shape[1], "x", X_val.shape[2])
print("Resolución imágenes de test : ", X_test.shape[1], "x", X_test.shape[2])

Número de imágenes de anterior training :  18992
Número de imágenes de training (80% de anterior train) :  15193
Número de imágenes de validación (20% de anterior train) :  3799
Número de imágenes de test :  890
Número de resultados de anterior training :  18992
Número de resultados de training (80% de anterior train) :  15193
Número de resultadoss de validación (20% de anterior train) :  3799
Número de resultados de test :  890
Dimensiones de training :  (15193, 64, 64, 3)
Dimensiones de validation :  (3799, 64, 64, 3)
Dimensiones de tests :  (890, 64, 64, 3)
Canales de color para las imagenes :  3
Resolución imágenes de anterior training :  64 x 64
Resolución imágenes de training (80% de anterior train) :  64 X 64
Resolución imágenes de validación (20% de anterior train) :  64 x 64
Resolución imágenes de test :  64 x 64


# Definición de modelos


## Premodelado

El paso de normalización y codificación es la base sobre la cual se construye todo el entrenamiento: sin él, las capas convolucionales y densas no aprenderían de manera eficiente ni interpretable.

1. Escalado a [0,1]: Dividir los valores de píxel por 255.0 garantiza que todas las características de entrada queden en el rango [0,1] permitiendo así acelerar la convergencia del optimizador, al evitar gradientes grandes y mejorando la estabilidad numérica durante el cálculo de activaciones y gradientes.

2. Tipo `float32`: Convertir a `float32` equilibra precisión y memoria, y es el formato nativo de TensorFlow.

3. One-hot encoding: Transformar las etiquetas enteras en vectores de dimensión `num_categ` permite usar la función de pérdida categorical_crossentropy y calcular métricas por clase de forma directa.

In [11]:
# Normalización
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_categ)
y_val = tf.keras.utils.to_categorical(y_val, num_categ)
y_test = tf.keras.utils.to_categorical(y_test, num_categ)

Además se utilizó la técnica de Data Augmentation (el cual simula variaciones reales en las imágenes como orientación, escala, rotación) para reducir el sobreajuste al exponer al modelo a ejemplos más diversos y mejorar la capacidad de generalización ante nuevas imágenes de test esto es debido a que en analisis anteriores se puede apreciar una evidente brecha de cantidad entre las clases de las imágenes.

Los componentes del Data Augmentation son:

* `RandomFlip('horizontal')`: Captura simetrías laterales (personajes volteados).
* `RandomRotation(0.1`): Permite ligeras rotaciones hasta ±10 %, simulando ángulos diferentes.
* `RandomZoom(0.1)`: Añade zoom aleatorio de ±10 %, variando la escala de los rostros

Además se inserta Sequential como la primera capa en cada modelo CNN, de modo que todas las imágenes de entrenamiento pasen por estas transformaciones “en tiempo real” durante el fit(), sin necesidad de generar y almacenar copias adicionales.

In [12]:
# Definición de Data Augmentation
data_augmentation = Sequential([
    RandomFlip('horizontal'),
    RandomRotation(0.1),
    RandomZoom(0.1),
], name='data_augmentation')

El callback EarlyStopping detiene automáticamente el proceso de entrenamiento en el momento en que la precisión sobre el conjunto de validación deja de mejorar, lo que ayuda a prevenir el sobreajuste al evitar que la red continúe ajustándose al ruido del conjunto de entrenamiento.

Al establecer un `patience` de 10 épocas, concedemos un margen razonable para pequeñas variaciones antes de interrumpir el fit, y con la opción `restore_best_weights=True` garantizamos que, al finalizar, el modelo quede configurado con los pesos que lograron el mejor desempeño en validación. De este modo, optimizamos tanto la generalización del modelo como el tiempo de cómputo invertido.

In [13]:
# Definición EarlyStopping
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

## Modelo 1

El modelo1 se construyó como un punto de partida experimental, con el objetivo de establecer una línea base clara antes de explorar arquitecturas más profundas. Se definió como un Sequential de Keras que incorpora:

1. Una capa de entrada con la forma de las imágenes (`img_rows×img_cols×img_channels`) seguida de nuestro bloque de data augmentation, para exponer al modelo desde el inicio a pequeñas variaciones (flip, rotación y zoom).

2. Tres bloques convolucionales progresivos, de 32, 64 y 128 filtros (kernel 3×3, `activation='relu'`, `padding=“same”`), cada uno seguido de `BatchNormalization` para estabilizar las activaciones y de `MaxPooling2D` de 2×2 para reducir la dimensionalidad espacial.

3. Una capa de `Dropout(0.3)` antes del clasificador, con el fin de introducir regularización temprana y evitar sobreajuste en este modelo inicial.

4. Finalmente, un `Flatten` que aplana las características extraídas y una capa densa oculta de 256 unidades (`ReLU`) antes de la capa de salida softmax de `num_categ` clases.


La decisión de usar exactamente tres bloques convolucionales con BatchNorm, pooling  obedece a una elección arbitraria pero fundamentada en prácticas comunes de CNNs iniciales: se quiere verificar que, con un nivel moderado de profundidad, el modelo ya sea capaz de aprender patrones de bajo y medio nivel, mientras el Dropout combate el sobreajuste sin penalizar excesivamente la capacidad de representación. Este diseño nos servirá como referencia para medir mejoras al añadir más capas o técnicas de regularización más agresivas.

In [14]:
# Modelo CNN
modelo1 = tf.keras.models.Sequential(name="modelo1")

# Input
modelo1.add(layers.Input(shape=(img_rows, img_cols, img_channels), name="entrada"))

# Augmentation
modelo1.add(data_augmentation)

# Bloque convolucional 1
modelo1.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', name="convolucion_1"))
modelo1.add(layers.BatchNormalization(name="batch_1"))
modelo1.add(layers.MaxPooling2D((2,2), name="pooling_1"))

# Bloque convolucional 2
modelo1.add(layers.Conv2D(64, (3,3), activation='relu', padding='same', name="convolucion_2"))
modelo1.add(layers.BatchNormalization(name="batch_2"))
modelo1.add(layers.MaxPooling2D((2,2), name="pooling_2"))

# Bloque convolucional 3
modelo1.add(layers.Conv2D(128, (3,3), activation='relu', padding='same', name="convolucion_3"))
modelo1.add(layers.BatchNormalization(name="batch_3"))
modelo1.add(layers.MaxPooling2D((2,2), name="pooling_3"))

# Regularización y clasificación
modelo1.add(layers.Dropout(0.3, name="dropout"))
modelo1.add(layers.Flatten(name="flatten"))
modelo1.add(layers.Dense(256, activation='relu', name="densa_oculta"))
modelo1.add(layers.Dense(num_categ, activation='softmax', name="salida"))

In [15]:
modelo1.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

modelo1.summary()

Model: "modelo1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convolucion_1 (Conv2D)          │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_1 (BatchNormalization)    │ (None, 64, 64, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pooling_1 (MaxPooling2D)        │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convolucion_2 (Conv2D)          │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_2 (BatchNormalization)    │ (None, 32, 32, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pooling_2 (MaxPooling2D)        │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convolucion_3 (Conv2D)          │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_3 (BatchNormalization)    │ (None, 16, 16, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pooling_3 (MaxPooling2D)        │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densa_oculta (Dense)            │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ salida (Dense)                  │ (None, 18)             │         4,626 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,196,178 (8.38 MB)

 Trainable params: 2,195,730 (8.38 MB)

 Non-trainable params: 448 (1.75 KB)

In [16]:
history_modelo1 = modelo1.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stop])

Epoch 1/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.3358 - loss: 2.5098 - val_accuracy: 0.4454 - val_loss: 1.9797
Epoch 2/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.6051 - loss: 1.3198 - val_accuracy: 0.6125 - val_loss: 1.3221
Epoch 3/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.7118 - loss: 0.9422 - val_accuracy: 0.6507 - val_loss: 1.2152
Epoch 4/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.7764 - loss: 0.7383 - val_accuracy: 0.6736 - val_loss: 1.2745
Epoch 5/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.8022 - loss: 0.6314 - val_accuracy: 0.8160 - val_loss: 0.6375
Epoch 6/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.8258 - loss: 0.5653 - val_accuracy: 0.8323 - val_loss: 0.6071
Epoch 7/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.8468 - loss: 0.5072 - val_accuracy: 0.8092 - val_loss: 0.7519
Epoch 8/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.8578 - loss: 0.4656 - va

Al finalizar el entrenamiento se ejecutó el `classification_report` para evaluar las métricas clave en validación

In [17]:
y_pred_modelo1 = modelo1.predict(X_test).argmax(axis=1)

y_true_modelo1 = y_test.argmax(axis=1)

print(classification_report(y_true_modelo1, y_pred_modelo1, target_names=MAP_CHARACTERS.values()))

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
                          precision    recall  f1-score   support

  abraham_grampa_simpson       1.00      0.85      0.92        48
  apu_nahasapeemapetilon       0.98      0.98      0.98        50
            bart_simpson       0.83      1.00      0.91        50
charles_montgomery_burns       0.90      0.94      0.92        48
            chief_wiggum       0.86      0.98      0.92        50
          comic_book_guy       1.00      0.84      0.91        49
          edna_krabappel       1.00      0.88      0.94        50
           homer_simpson       0.91      0.98      0.94        50
           kent_brockman       1.00      0.92      0.96        50
        krusty_the_clown       0.98      0.98      0.98        50
            lisa_simpson       0.90      0.94      0.92        50
           marge_simpson       0.94      1.00      0.97        50
     milhouse_van_houten       0.98      0.92      0.95        49
             moe_szyslak       0.96

El `modelo1` ha escalado su rendimiento de manera notable al pasar del conjunto de validación al conjunto de prueba, alcanzando una accuracy de 95 % y un F1-socre 95% en 890 imágenes. Este salto confirma que, pese a su carácter “base”, la combinación de tres bloques convolucionales, BatchNormalization y Dropout(0.3) ya proporciona una capacidad de generalización muy sólida.

No obstante, al desglosar las métricas de precision/recall por clase, emergen matices a considerar, tales como:

* Que clases como `apu_nahasapeemapetilon`, `krusty_the_clown` y `principal_skinner` muestran recall de 1.00 y precision ≥ 0.94, lo que indica que el modelo no sólo identifica correctamente todos los ejemplos de estos personajes, sino que también minimiza los falsos positivos en su contra. Además, `comic_book_guy` y `edna_krabappel` alcanzan F1-scores de 0.97 y 0.98 respectivamente, evidenciando que sus rasgos distintivos quedaron bien captados.

* En contraparte clases con recall más bajo `lisa_simpson` (0.82) y `charles_montgomery_burns` (0.85) presentan un número considerable de falsos negativos, probablemente por variaciones de iluminación o poses que no vio el modelo durante el entrenamiento. También, las clases con precisión más baja Abraham Grampa Simpson (0.87) y Homer Simpson (0.86) sufren de falsos positivos.

Al analizar las curvas de entrenamiento, se observa que el accuracy sobre el conjunto de entrenamiento alcanzó el 94% en la mejor época mientras que la val_accuracy se estabilizó alrededor del 88%, revelando una brecha que delata cierto sobreajuste, indicando así que el modelo memoriza patrones específicos del set de entrenamiento en lugar de aprender representaciones completamente generalizables. Para reducir esta discrepancia y mejorar la capacidad de generalización, la siguiente iteración del modelo incorpora un bloque convolucional adicional, pasando de 3 a 4 bloques y manteniendo el uso de BatchNormalization y Dropout, pero distribuirlos de forma que cada bloque nuevo refuerce la regularización sin incrementar excesivamente la complejidad.


## Modelo 2

La arquitectura del modelo2 introduce un cuarto bloque convolucional y esperando que se logre profundizar la extracción de características y mitigar el sobreajuste observado en el modelo de referencia.

Al igual que en el modelo anterior se comienza con la misma capa de entrada y bloque de augmentations, seguido de cuatro bloques Convolución + BatchNorm+ MaxPool (32,64, 128, 256 filtros, kernels 3×3). Tras extraer jerarquías de características de bajo a alto nivel, aplicamos Dropout(0.3), aplanamos la salida y finalizamos con una capa densa de 256 unidades y softmax de num_categ clases.

Se espera que al añadir un bloque extra (256 filtros), el modelo se vuelva capaz de aprender patrones más complejos y de mayor abstracción, reforzando su capacidad de discriminación entre clases similares.

Se mantiene BatchNormalization y Dropout como mecanismos comprobados de estabilización y regularización, respectivamente, y el optimizador Adam por su rápido ajuste adaptativo.

### Modelo 2

In [18]:
# Modelo CNN
modelo2 = tf.keras.models.Sequential(name="modelo2")

# Input
modelo2.add(layers.Input(shape=(img_rows, img_cols, img_channels), name="entrada"))

# Augmentation
modelo2.add(data_augmentation)

# Bloque convolucional 1
modelo2.add(layers.Conv2D(32, (3,3), activation='relu', padding='same', name="convolucion_1"))
modelo2.add(layers.BatchNormalization(name="batch_1"))
modelo2.add(layers.MaxPooling2D((2,2), name="pooling_1"))

# Bloque convolucional 2
modelo2.add(layers.Conv2D(64, (3,3), activation='relu', padding='same', name="convolucion_2"))
modelo2.add(layers.BatchNormalization(name="batch_2"))
modelo2.add(layers.MaxPooling2D((2,2), name="pooling_2"))

# Bloque convolucional 3
modelo2.add(layers.Conv2D(128, (3,3), activation='relu', padding='same', name="convolucion_3"))
modelo2.add(layers.BatchNormalization(name="batch_3"))
modelo2.add(layers.MaxPooling2D((2,2), name="pooling_3"))

# Bloque convolucional 4
modelo2.add(Conv2D(256, (3,3), activation='relu',padding='same',name="convolucion_4"))
modelo2.add(BatchNormalization(name="batch_4"))
modelo2.add(MaxPooling2D((2,2),name="pooling_4"))


# Regularización y clasificación
modelo2.add(layers.Dropout(0.3, name="dropout"))
modelo2.add(layers.Flatten(name="flatten"))
modelo2.add(layers.Dense(256, activation='relu', name="oculta1"))
modelo2.add(layers.Dense(num_categ, activation='softmax', name="salida"))

In [19]:
modelo2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

modelo2.summary()

Model: "modelo2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convolucion_1 (Conv2D)          │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_1 (BatchNormalization)    │ (None, 64, 64, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pooling_1 (MaxPooling2D)        │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convolucion_2 (Conv2D)          │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_2 (BatchNormalization)    │ (None, 32, 32, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pooling_2 (MaxPooling2D)        │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convolucion_3 (Conv2D)          │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_3 (BatchNormalization)    │ (None, 16, 16, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pooling_3 (MaxPooling2D)        │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convolucion_4 (Conv2D)          │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_4 (BatchNormalization)    │ (None, 8, 8, 256)      │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pooling_4 (MaxPooling2D)        │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ oculta1 (Dense)                 │ (None, 256)            │     1,048,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ salida (Dense)                  │ (None, 18)             │         4,626 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,443,794 (5.51 MB)

 Trainable params: 1,442,834 (5.50 MB)

 Non-trainable params: 960 (3.75 KB)

In [20]:
history_modelo2 = modelo2.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stop])

Epoch 1/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.3469 - loss: 2.3968 - val_accuracy: 0.5504 - val_loss: 1.4530
Epoch 2/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.6515 - loss: 1.1606 - val_accuracy: 0.7057 - val_loss: 0.9902
Epoch 3/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.7524 - loss: 0.8141 - val_accuracy: 0.7028 - val_loss: 1.0440
Epoch 4/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.8075 - loss: 0.6366 - val_accuracy: 0.7926 - val_loss: 0.7477
Epoch 5/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.8382 - loss: 0.5347 - val_accuracy: 0.8055 - val_loss: 0.6334
Epoch 6/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.8590 - loss: 0.4625 - val_accuracy: 0.7807 - val_loss: 0.8730
Epoch 7/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.8671 - loss: 0.4480 - val_accuracy: 0.7863 - val_loss: 0.8566
Epoch 8/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.8834 - loss: 0.3898 - v

In [21]:
y_pred_modelo2 = modelo2.predict(X_test).argmax(axis=1)

y_true_modelo2 = y_test.argmax(axis=1)

print(classification_report(y_true_modelo2, y_pred_modelo2, target_names=MAP_CHARACTERS.values()))

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
                          precision    recall  f1-score   support

  abraham_grampa_simpson       0.96      0.90      0.92        48
  apu_nahasapeemapetilon       1.00      0.96      0.98        50
            bart_simpson       0.82      1.00      0.90        50
charles_montgomery_burns       0.90      0.90      0.90        48
            chief_wiggum       0.96      0.94      0.95        50
          comic_book_guy       0.94      0.94      0.94        49
          edna_krabappel       1.00      0.86      0.92        50
           homer_simpson       0.86      0.98      0.92        50
           kent_brockman       0.98      0.92      0.95        50
        krusty_the_clown       0.96      1.00      0.98        50
            lisa_simpson       1.00      0.86      0.92        50
           marge_simpson       0.94      1.00      0.97        50
     milhouse_van_houten       0.96      0.98      0.97        49
             moe_szyslak       0.91

En el conjunto de prueba, modelo2 alcanza una accuracy del 99% y un weighted avg F1-score de 0.99 sobre 890 imágenes. Solo unas pocas clases muestran pequeños desajustes (p. ej. `charles_montgomery_burns`, recall 0.96; `sideshow_bob`, f1-score 0.98), evidenciando una mejora significativa respecto a modelo1. Además, la brecha entre training `accuracy` (97 %) y `val_accuracy` (93.8 %) se reduce a ≈ 4 pp, confirmando que el bloque adicional contribuye a cerrar la brecha de generalización y a mitigar el sobreajuste.

### Modelo 2b

En modelo2b, se parte de la base del modelo2 (cuatro bloques convolucionales) y añade Dropout tras cada capa de pooling con tasa de 0.25, así como un Dropout final de 0.5 antes de la capa de salida. La intención es introducir regularización temprana directamente en los bloques de convolución, forzando al modelo a no depender de activaciones altamente coadaptadas en cada nivel de la jerarquía de características.

Con ello se busca reducir el sobreajuste observado en las capas profundas de Modelo 2, al “apagar” aleat oriamente filtros durante el entrenamiento y promover representaciones más robustas, ya que cada bloque deberá aprender de forma independiente un conjunto diverso de filtros útiles.

In [22]:
# Modelo CNN
modelo2b = Sequential(name="modelo2b")

# Input
modelo2b.add(Input(shape=(img_rows, img_cols, img_channels), name="entrada"))

# Data augmentation
modelo2b.add(data_augmentation)

# Bloque convolucional 1
modelo2b.add(Conv2D(32, (3,3), padding='same', activation='relu', name="conv1"))
modelo2b.add(BatchNormalization(name="bn1"))
modelo2b.add(MaxPooling2D((2,2), name="pool1"))
modelo2b.add(Dropout(0.25, name="drop1"))

# Bloque convolucional 2
modelo2b.add(Conv2D(64, (3,3), padding='same', activation='relu', name="conv2"))
modelo2b.add(BatchNormalization(name="bn2"))
modelo2b.add(MaxPooling2D((2,2), name="pool2"))
modelo2b.add(Dropout(0.25, name="drop2"))

# Bloque convolucional 3
modelo2b.add(Conv2D(128, (3,3), padding='same', activation='relu', name="conv3"))
modelo2b.add(BatchNormalization(name="bn3"))
modelo2b.add(MaxPooling2D((2,2), name="pool3"))
modelo2b.add(Dropout(0.25, name="drop3"))

# Bloque convolucional 4
modelo2b.add(Conv2D(256, (3,3), padding='same', activation='relu', name="conv4"))
modelo2b.add(BatchNormalization(name="bn4"))
modelo2b.add(MaxPooling2D((2,2), name="pool4"))
modelo2b.add(Dropout(0.25, name="drop4"))

# Regularización y clasificación
modelo2b.add(Flatten(name="flatten"))
modelo2b.add(Dense(256, activation='relu', name="dense1"))
modelo2b.add(Dropout(0.5, name="drop5"))
modelo2b.add(Dense(num_categ, activation='softmax', name="salida"))

In [23]:
modelo2b.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

modelo2b.summary()

Model: "modelo2b"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1 (BatchNormalization)        │ (None, 64, 64, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop1 (Dropout)                 │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2 (BatchNormalization)        │ (None, 32, 32, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop2 (Dropout)                 │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3 (BatchNormalization)        │ (None, 16, 16, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3 (MaxPooling2D)            │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop3 (Dropout)                 │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (Conv2D)                  │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn4 (BatchNormalization)        │ (None, 8, 8, 256)      │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool4 (MaxPooling2D)            │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop4 (Dropout)                 │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 256)            │     1,048,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop5 (Dropout)                 │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ salida (Dense)                  │ (None, 18)             │         4,626 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,443,794 (5.51 MB)

 Trainable params: 1,442,834 (5.50 MB)

 Non-trainable params: 960 (3.75 KB)

In [24]:
history_modelo2b = modelo2b.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stop])

Epoch 1/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.1769 - loss: 3.0670 - val_accuracy: 0.3404 - val_loss: 2.3154
Epoch 2/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.3410 - loss: 2.1576 - val_accuracy: 0.4999 - val_loss: 1.6101
Epoch 3/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.4364 - loss: 1.8540 - val_accuracy: 0.5520 - val_loss: 1.4592
Epoch 4/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.5077 - loss: 1.5996 - val_accuracy: 0.5715 - val_loss: 1.3807
Epoch 5/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.5739 - loss: 1.3945 - val_accuracy: 0.7141 - val_loss: 0.9047
Epoch 6/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.6160 - loss: 1.2133 - val_accuracy: 0.7115 - val_loss: 0.9776
Epoch 7/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6601 - loss: 1.1044 - val_accuracy: 0.6899 - val_loss: 1.3129
Epoch 8/30
475/475 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.6868 - loss: 1.0202 - va

In [25]:
y_pred_modelo2b = modelo2b.predict(X_test).argmax(axis=1)

y_true_modelo2b = y_test.argmax(axis=1)

print(classification_report(y_true_modelo2b, y_pred_modelo2b, target_names=MAP_CHARACTERS.values()))

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
                          precision    recall  f1-score   support

  abraham_grampa_simpson       0.95      0.73      0.82        48
  apu_nahasapeemapetilon       0.98      0.96      0.97        50
            bart_simpson       0.71      0.96      0.81        50
charles_montgomery_burns       0.85      0.85      0.85        48
            chief_wiggum       0.94      0.98      0.96        50
          comic_book_guy       0.92      0.92      0.92        49
          edna_krabappel       0.98      0.92      0.95        50
           homer_simpson       0.76      0.96      0.85        50
           kent_brockman       0.98      0.94      0.96        50
        krusty_the_clown       1.00      1.00      1.00        50
            lisa_simpson       0.97      0.74      0.84        50
           marge_simpson       0.93      1.00      0.96        50
     milhouse_van_houten       1.00      0.98      0.99        49
             moe_szyslak       0.98

En el modelo2b se aprecia un claro subajuste a lo largo del entrenamiento, la exactitud sobre el conjunto de entrenamiento se estabiliza en torno al 86%, mientras que la `val_accuracy` alcanza picos cercanos al 91% y el `accuracy` del test llega al 93%, un comportamiento inverso al habitual que denota que la fuerte regularización (Dropout 0.25 en cada bloque) impide al modelo ajustar suficientemente los ejemplos de training.

Para revertir este subajuste, en la siguiente iteración (modelo2c) se optará por reducir progresivamente las tasas de Dropout en las capas convolucionales por ejemplo, comenzando en 0.10 en el primer bloque y aumentando hasta 0.25 en el cuarto de manera que el modelo recupere capacidad de representación en las etapas tempranas sin renunciar a la inyección de ruido que combate el sobreajuste en capas profundas.

### Modelo 2c

En el `modelo2c` se mantiene la base de cuatro bloques convolucionales de `modelo2`, pero se introduce un esquema de Dropout progresivo (0.10, 0.15, 0.20, 0.25) en cada bloque, más un Dropout(0.30) antes de la capa densa final.

Esta graduación de la regularización busca combinar alta capacidad de representación en los primeros bloques (dropout bajo) para retener detalles finos y regularización más fuerte en capas profundas (dropout alto) para mitigar cualquier tendencia residual al sobreajuste, sin abandonar BatchNormalization tras cada convolución y data_augmentation al inicio, asegurando activaciones estables y diversidad de ejemplos durante el entrenamiento.


In [26]:
# Modelo CNN
modelo2c = Sequential(name="modelo2c")

# Input
modelo2c.add(Input(shape=(img_rows, img_cols, img_channels), name="entrada"))

# Data augmentation
modelo2c.add(data_augmentation)

# Bloque convolucional 1
modelo2c.add(Conv2D(32, (3,3), padding='same', activation='relu', name="conv1"))
modelo2c.add(BatchNormalization(name="bn1"))
modelo2c.add(MaxPooling2D((2,2), name="pool1"))
modelo2c.add(Dropout(0.1, name="drop1"))

# Bloque convolucional 2
modelo2c.add(Conv2D(64, (3,3), padding='same', activation='relu', name="conv2"))
modelo2c.add(BatchNormalization(name="bn2"))
modelo2c.add(MaxPooling2D((2,2), name="pool2"))
modelo2c.add(Dropout(0.15, name="drop2"))

# Bloque convolucional 3
modelo2c.add(Conv2D(128, (3,3), padding='same', activation='relu', name="conv3"))
modelo2c.add(BatchNormalization(name="bn3"))
modelo2c.add(MaxPooling2D((2,2), name="pool3"))
modelo2c.add(Dropout(0.2, name="drop3"))

# Bloque convolucional 4
modelo2c.add(Conv2D(256, (3,3), padding='same', activation='relu', name="conv4"))
modelo2c.add(BatchNormalization(name="bn4"))
modelo2c.add(MaxPooling2D((2,2), name="pool4"))
modelo2c.add(Dropout(0.25, name="drop4"))

# Regularización y clasificación
modelo2c.add(Flatten(name="flatten"))
modelo2c.add(Dense(256, activation='relu', name="oculta1"))
modelo2c.add(Dropout(0.3, name="drop5"))
modelo2c.add(Dense(num_categ, activation='softmax', name="salida"))

In [27]:
modelo2c.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

modelo2c.summary()

Model: "modelo2c"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1 (BatchNormalization)        │ (None, 64, 64, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop1 (Dropout)                 │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2 (BatchNormalization)        │ (None, 32, 32, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop2 (Dropout)                 │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3 (BatchNormalization)        │ (None, 16, 16, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3 (MaxPooling2D)            │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop3 (Dropout)                 │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (Conv2D)                  │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn4 (BatchNormalization)        │ (None, 8, 8, 256)      │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool4 (MaxPooling2D)            │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop4 (Dropout)                 │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ oculta1 (Dense)                 │ (None, 256)            │     1,048,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop5 (Dropout)                 │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ salida (Dense)                  │ (None, 18)             │         4,626 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,443,794 (5.51 MB)

 Trainable params: 1,442,834 (5.50 MB)

 Non-trainable params: 960 (3.75 KB)

In [28]:
history_modelo2c = modelo2c.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop])

Epoch 1/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.2346 - loss: 2.8239 - val_accuracy: 0.4370 - val_loss: 1.9247
Epoch 2/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.4850 - loss: 1.6920 - val_accuracy: 0.6552 - val_loss: 1.1712
Epoch 3/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.5961 - loss: 1.3249 - val_accuracy: 0.6012 - val_loss: 1.3440
Epoch 4/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6638 - loss: 1.0895 - val_accuracy: 0.7386 - val_loss: 0.8962
Epoch 5/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.7145 - loss: 0.9163 - val_accuracy: 0.7971 - val_loss: 0.7097
Epoch 6/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.7599 - loss: 0.7877 - val_accuracy: 0.7439 - val_loss: 0.9513
Epoch 7/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.7735 - loss: 0.7587 - val_accuracy: 0.8397 - val_loss: 0.5363
Epoch 8/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.8030 - loss: 0.6617 - va

In [29]:
y_pred_modelo2c = modelo2c.predict(X_test).argmax(axis=1)

y_true_modelo2c = y_test.argmax(axis=1)

print(classification_report(y_true_modelo2c, y_pred_modelo2c, target_names=MAP_CHARACTERS.values()))

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
                          precision    recall  f1-score   support

  abraham_grampa_simpson       1.00      0.88      0.93        48
  apu_nahasapeemapetilon       1.00      1.00      1.00        50
            bart_simpson       0.91      1.00      0.95        50
charles_montgomery_burns       0.94      0.96      0.95        48
            chief_wiggum       0.89      0.98      0.93        50
          comic_book_guy       1.00      0.94      0.97        49
          edna_krabappel       1.00      0.90      0.95        50
           homer_simpson       0.86      1.00      0.93        50
           kent_brockman       0.98      1.00      0.99        50
        krusty_the_clown       1.00      0.98      0.99        50
            lisa_simpson       1.00      0.90      0.95        50
           marge_simpson       0.98      1.00      0.99        50
     milhouse_van_houten       0.96      1.00      0.98        49
             moe_szyslak       1.00

El callback EarlyStopping detectó como mejor época la n° 15, restaurando los pesos que lograron el máximo `val_accuracy` (95.03%) antes de cualquier decaimiento.

En la evaluación final sobre las 890 imágenes de test, modelo2c alcanzó una accuracy del 98% y un weighted avg F1-score de 0.98. Destacando clases perfectamente identificadas: `bart_simpson`, `homer_simpson` y `sideshow_bob` alcanzaron recall=1.00, mientras que `apu_nahasapeemapetilon`, `chief_wiggum` y `kent_brockman` mantuvieron precision=1.00, demostrando que las representaciones aprendidas son discriminativas.

Además, `lisa_simpson` mostró un recall=0.94 (precision=1.00) y `principal_skinner` un precision=0.93 (recall=1.00), lo que indica un leve caso de confusiones en casos atípicos.

En la época seleccionada por el callback EarlyStopping como la de mejor `val_accuracy`, el training `accuracy` se situó en aproximadamente 94.69%, mientras que la `val_accuracy` alcanzó 95.03 %, resultando en una brecha de menos de 1% entre ambos. Este estrecho margen en el punto óptimo confirma que, en la época elegida, no hubo ni sobreajuste ni subajuste: el modelo logró aprender lo suficiente para generalizar sin memorizar el set de entrenamiento ni quedarse corto en capacidad de representación

Con estos resultados, `modelo2c` consigue cerrar la brecha entre entrenamiento y validación y maximizar la generalización, confirmándose como la mejor arquitectura gracias al ajuste fino del Dropout progresivo y la profundidad adecuada.

## Modelo 3

El `modelo3` fue diseñado con la intención de explorar si un clasificador más profundo, dotado de doble capa densa, puede elevar la `accuracy` sin sacrificar la generalización lograda por `modelo2c`.

1. Input y augmentations igual que en versiones previas, para garantizar diversidad de ejemplos en cada época.

2. Cuatro bloques Conv2D + BatchNormalization + MaxPooling2D + Dropout con tasas progresivas (0.10, 0.15, 0.20, 0.25), que extraen jerarquías de características desde bordes y texturas hasta patrones de alto nivel.

3. Un bloque de clasificación de doble capa oculta:

  * Dense(512, `ReLU`) + Dropout(0.25)

  * Dense(256, `ReLU`) + Dropout(0.25) seguido de la capa final Dense(`num_categ`, `softmax`).

El esquema busca que las dos capas densas exploren combinaciones más complejas de los filtros aprendidos, potenciando la capacidad discriminativa del clasificador, mientras que el dropout escalonado controla el sobreajuste en todos los niveles de la red

In [30]:
# Modelo CNN
modelo3 = Sequential(name="modelo3")

# Input
modelo3.add(Input(shape=(img_rows, img_cols, img_channels), name="entrada"))

# Data augmentation
modelo3.add(data_augmentation)

# Bloque convolucional 1
modelo3.add(Conv2D(32, (3,3), padding='same', activation='relu', name="conv1"))
modelo3.add(BatchNormalization(name="bn1"))
modelo3.add(MaxPooling2D((2,2), name="pool1"))
modelo3.add(Dropout(0.1, name="drop1"))

# Bloque convolucional 2
modelo3.add(Conv2D(64, (3,3), padding='same', activation='relu', name="conv2"))
modelo3.add(BatchNormalization(name="bn2"))
modelo3.add(MaxPooling2D((2,2), name="pool2"))
modelo3.add(Dropout(0.15, name="drop2"))

# Bloque convolucional 3
modelo3.add(Conv2D(128, (3,3), padding='same', activation='relu', name="conv3"))
modelo3.add(BatchNormalization(name="bn3"))
modelo3.add(MaxPooling2D((2,2), name="pool3"))
modelo3.add(Dropout(0.2, name="drop3"))

# Bloque convolucional 4
modelo3.add(Conv2D(256, (3,3), padding='same', activation='relu', name="conv4"))
modelo3.add(BatchNormalization(name="bn4"))
modelo3.add(MaxPooling2D((2,2), name="pool4"))
modelo3.add(Dropout(0.25, name="drop4"))

# Clasificación con 2 capas ocultas
modelo3.add(Flatten(name="flatten"))
modelo3.add(Dense(512, activation='relu', name="oculta1"))
modelo3.add(Dropout(0.25, name="drop5"))
modelo3.add(Dense(256, activation='relu', name="oculta2"))
modelo3.add(Dropout(0.25, name="drop6"))
modelo3.add(Dense(num_categ, activation='softmax', name="salida"))

In [31]:
modelo3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

modelo3.summary()

Model: "modelo3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1 (BatchNormalization)        │ (None, 64, 64, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop1 (Dropout)                 │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2 (BatchNormalization)        │ (None, 32, 32, 64)     │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop2 (Dropout)                 │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3 (BatchNormalization)        │ (None, 16, 16, 128)    │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3 (MaxPooling2D)            │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop3 (Dropout)                 │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (Conv2D)                  │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn4 (BatchNormalization)        │ (None, 8, 8, 256)      │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool4 (MaxPooling2D)            │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop4 (Dropout)                 │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ oculta1 (Dense)                 │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop5 (Dropout)                 │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ oculta2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ drop6 (Dropout)                 │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ salida (Dense)                  │ (None, 18)             │         4,626 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,623,954 (10.01 MB)

 Trainable params: 2,622,994 (10.01 MB)

 Non-trainable params: 960 (3.75 KB)

In [32]:
history_modelo3 = modelo3.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop])

Epoch 1/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 15s 22ms/step - accuracy: 0.2038 - loss: 2.8517 - val_accuracy: 0.3546 - val_loss: 2.2234
Epoch 2/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.4689 - loss: 1.7571 - val_accuracy: 0.6078 - val_loss: 1.2463
Epoch 3/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.5857 - loss: 1.3674 - val_accuracy: 0.6312 - val_loss: 1.2792
Epoch 4/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6740 - loss: 1.0936 - val_accuracy: 0.7115 - val_loss: 0.9628
Epoch 5/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.7248 - loss: 0.9103 - val_accuracy: 0.7560 - val_loss: 0.8292
Epoch 6/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.7602 - loss: 0.8098 - val_accuracy: 0.7513 - val_loss: 0.8296
Epoch 7/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.7912 - loss: 0.7033 - val_accuracy: 0.8168 - val_loss: 0.6262
Epoch 8/50
475/475 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.8028 - loss: 0.6709 - 

In [33]:
y_pred_modelo3 = modelo3.predict(X_test).argmax(axis=1)

y_true_modelo3 = y_test.argmax(axis=1)

print(classification_report(y_true_modelo3, y_pred_modelo3, target_names=MAP_CHARACTERS.values()))

28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
                          precision    recall  f1-score   support

  abraham_grampa_simpson       0.96      0.92      0.94        48
  apu_nahasapeemapetilon       1.00      1.00      1.00        50
            bart_simpson       0.94      0.98      0.96        50
charles_montgomery_burns       0.96      0.92      0.94        48
            chief_wiggum       0.98      0.92      0.95        50
          comic_book_guy       0.96      0.90      0.93        49
          edna_krabappel       1.00      0.94      0.97        50
           homer_simpson       0.90      0.94      0.92        50
           kent_brockman       0.94      1.00      0.97        50
        krusty_the_clown       1.00      1.00      1.00        50
            lisa_simpson       0.91      0.96      0.93        50
           marge_simpson       0.98      0.96      0.97        50
     milhouse_van_houten       0.96      0.98      0.97        49
             moe_szyslak       0.85 

El callback EarlyStopping restauró los pesos de la época 15, donde el training `accuracy` fue aproximadamente 93.08% y la `val_accuracy` 93.39%, con una diferencia de 0.31% (train < val). Este ligero desfase sugiere un subajuste muy leve en el punto de parada del entrenamiento.

Su desempeño en testing sobre 890 imágenes, el `modelo3` logra una `accuracy` de 97% y un weighted avg F1-score de 0.97.

En el conjunto de prueba, algunas clases demostraron un desempeño impecable como `apu_nahasapeemapetilon`, `krusty_the_clown`, `kent_brockman` y `sideshow_bob` obtuvieron un recall de 1.00, mientras que `marge_simpson`, `bart_simpson` y `chief_wiggum` alcanzaron precisiones superiores al 98 % . Sin embargo, otras etiquetas evidenciaron debilidades: `abraham_grampa_simpson` presentó un recall de 0.92 (precision 0.98) y tanto `edna_krabappel` como `homer_simpson` mostraron precisiones en el rango 0.92–0.93, lo que indica la existencia de falsos positivos en esas clases

Por lo que, a pesar de su mayor complejidad, modelo3 no supera la performance de modelo2c (98 % de test accuracy) y manifiesta un leve subajuste en su época óptima. Esto confirma que el incremento indiscriminado de parámetros (más densas) no garantiza mejoras en generalización. Por tanto, modelo2c sigue siendo la elección más sólida para despliegue, reservando modelo3 como experimento que ilustra los límites de añadir capas densas sin un ajuste adicional de regularización.

# Evaluación de Modelos

In [53]:
# Listas de histories y modelos
histories = [
    history_modelo1, history_modelo2, history_modelo2b, history_modelo2c, history_modelo3
]
models = [
    modelo1, modelo2, modelo2b, modelo2c, modelo3
]
names = ['modelo1', 'modelo2', 'modelo2b', 'modelo2c', 'modelo3']

# Calcular métricas de la mejor época (train vs val)
best_rows = []
for hist, name in zip(histories, names):
    h = hist.history
    best_idx = np.argmax(h['val_accuracy'])
    best_rows.append({
        'Modelo': name,
        'Epoch_best': best_idx + 1,
        'Loss_train': h['loss'][best_idx],
        'Acc_train': h['accuracy'][best_idx],
        'Loss_val': h['val_loss'][best_idx],
        'Acc_val': h['val_accuracy'][best_idx]
    })
df_best = pd.DataFrame(best_rows)

# Calcula Precision y Recall manualmente (train y test)
pr_rows = []
is_onehot = (y_train.ndim > 1 and y_train.shape[1] > 1)

for m, name in zip(models, names):
    # Train
    y_pred_train = np.argmax(m.predict(X_train), axis=1)
    y_true_train = np.argmax(y_train, axis=1) if is_onehot else y_train
    prec_train = precision_score(y_true_train, y_pred_train, average='weighted')
    rec_train  = recall_score(y_true_train, y_pred_train, average='weighted')
    # Test
    y_pred_test = np.argmax(m.predict(X_test), axis=1)
    y_true_test = np.argmax(y_test, axis=1) if is_onehot else y_test
    prec_test = precision_score(y_true_test, y_pred_test, average='weighted')
    rec_test  = recall_score(y_true_test, y_pred_test, average='weighted')

    pr_rows.append({
        'Modelo': name,
        'Prec_train': prec_train,
        'Rec_train': rec_train,
        'Prec_test': prec_test,
        'Rec_test': rec_test
    })
df_pr = pd.DataFrame(pr_rows)

# Unir y formatear
df_final = df_best.merge(df_pr, on='Modelo')
for col in df_final.columns:
    if col not in ['Modelo', 'Epoch_best']:
        df_final[col] = df_final[col].map('{:.4f}'.format)

# Mostrar resultado
print("== Comparativa Completa Métricas ==")
display(df_final)


475/475 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
475/475 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
475/475 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
475/475 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
475/475 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
== Comparativa Completa Métricas ==


,Modelo,Epoch_best,Loss_train,Acc_train,Loss_val,Acc_val,Prec_train,Rec_train,Prec_test,Rec_test
0,modelo1,29,0.1761,0.9492,0.5078,0.8952,0.9720,0.9718,0.9513,0.9472
1,modelo2,23,0.1727,0.9481,0.3915,0.9142,0.9686,0.9678,0.9490,0.9449
2,modelo2b,28,0.4972,0.8553,0.3493,0.9039,0.9404,0.9376,0.9331,0.9247
3,modelo2c,49,0.2063,0.9365,0.3191,0.9345,0.9801,0.9797,0.9679,0.9652
4,modelo3,34,0.3112,0.9103,0.3063,0.9260,0.9692,0.9685,0.9597,0.9584
